# Author imformation:
Fan Shengzhe, Shanghaijiaotong University, Shanghai, China  
Email: fanshengzhe@sjtu.edu.cn

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
os.chdir("drive/My Drive/crop_identification2/crop_identification")

# Install suitable tensorflow version
Notice: This script is based on tensorflow 2.x, and tensorflow 1.x will not be supported.

In [ ]:
import tensorflow as tf 
print(tf.__version__)
# if tf.__version__ != '2.8.2':
#   !pip install tensorflow==2.8.2

2.9.2


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3202661005756027316
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14415560704
locality {
  bus_id: 1
  links {
  }
}
incarnation: 136644900925538041
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [ ]:
try:
  import tensorflow_addons
except:
  !pip install tensorflow_addons

# copy data to temporary folder

copy the data to the temporary folder in colab instead read the data from google drive may solve the problem which is extremely slow in the first epoch click the following URL for details: https://discuss.pytorch.org/t/drastically-slow-speed-at-first-epoch/12551/9

copying folder may be slow, zip the folder may solve this porblem 


In [ ]:
if not (os.path.exists('/content/data.rar') or os.path.exists('/content/data')):
  !cp data.rar /content/ 

In [ ]:
if not os.path.exists('/content/data'):
  !unrar x -inul /content/data.rar /content
  !rm -rf /content/data.rar

In [ ]:
DATA_ROOT_PATH = '/content/data'

# Import necessary modules

In [ ]:
import tensorflow as tf
import itertools
import datetime
import time
import json
import math
import io
from matplotlib import pyplot as plt
from functools import partial
from model_builder import *
from config import *
from utils import *
import gc

In [ ]:
BATCH_SIZE = 16

# Training

In [ ]:
@tf.function
def test_step(input_img, label):
    pred = model(input_img, training=False)
    loss = loss_obj(label, pred)
    top1_metrics_obj.update_state(label, pred)
    top3_metrics_obj.update_state(label, pred)
    top5_metrics_obj.update_state(label, pred)
    top10_metrics_obj.update_state(label, pred)
    return loss

if __name__ == "__main__":
    # Creat class dict
    if not os.path.exists(os.path.join(CLASS_DICT_DIR, "class_dict.json")):
      raise Exception("class dict not found") 
    else:
      with open(f"{CLASS_DICT_DIR}/class_dict.json", "r") as jfile:
          cls_dict, rev_cls_dict = json.load(jfile)
      print("INFO: class_dict found and reused")

    # All models
    # vanilla_cnn = ['vanilla_cnn']
    # ghost_efficientnet_v2 = ['ghost_efficientnet_v2_s', 'ghost_efficientnet_v2_m', 'ghost_efficientnet_v2_l', 'ghost_efficientnet_v2_xl']
    # efficientnet_v2 = ['efficientnet_v2_s', 'efficientnet_v2_m', 'efficientnet_v2_l', 'efficientnet_v2_xl']
    # mobilenet_v3 = ['mobilenet_v3_small', 'mobilenet_v3_large']
    # se_resnext = ['se_resnext_50', 'se_resnext_101']

    model_name = 'se_resnext_101'
    # Define model
    model = get_model(model_name, IMAGE_H, IMAGE_W, 3, len(cls_dict))
    
    # Data pipeline
    test_image_path_list, test_label = get_img_path_list(DATA_ROOT_PATH, TEST_DIR_NAME, class_dict=cls_dict, one_hot=True)
    test_dataset = tf.data.Dataset.from_tensor_slices((test_image_path_list, test_label))
    tsfm_test = partial(transform_test, height=IMAGE_H, width=IMAGE_W)
    test_dataset = test_dataset.map(tsfm_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    test_dataset = test_dataset.batch(BATCH_SIZE)
    test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    # Loss, metrices
    loss_obj = tf.keras.losses.CategoricalCrossentropy()
    top1_metrics_obj = tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='top_1_categorical_accuracy')
    top3_metrics_obj = tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top_3_categorical_accuracy')
    top5_metrics_obj = tf.keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_categorical_accuracy')
    top10_metrics_obj = tf.keras.metrics.TopKCategoricalAccuracy(k=10, name='top_10_categorical_accuracy')
    
    # Define ckpt
    best_ckpt = tf.train.Checkpoint(model=model)

    # Load ckpt
    if not os.path.exists(os.path.join(BEST_CHECKPOINT_DIR, model_name)):
      raise Exception("ckpt not found")
    else:
      best_ckpt.restore(tf.train.latest_checkpoint(os.path.join(BEST_CHECKPOINT_DIR, model_name))).expect_partial()

    test_total_loss = 0
    batch_num = 1
    top1_metrics_obj.reset_states()
    top3_metrics_obj.reset_states()
    top5_metrics_obj.reset_states()
    top10_metrics_obj.reset_states()
    starttime = time.time()
    print('model:', model_name)
    print('loss:', 'top1_metrics:', 'top3_metrics:', 'top5_metrics:', 
        'top10_metrics:', 'rwma_metrics:', 'time_taken(s):', 
        'sample_num', 'speed(iter/s)',)
    for n, (input_img, label) in test_dataset.enumerate():
        loss = test_step(input_img, label)
        test_total_loss += loss
        batch_num = n + 1
        print('\rtesting', 
          round(loss.numpy(), 4),
          round(top1_metrics_obj.result().numpy(), 4),
          round(top3_metrics_obj.result().numpy(), 4),
          round(top5_metrics_obj.result().numpy(), 4),
          round(top10_metrics_obj.result().numpy(), 4),
          round((10*top1_metrics_obj.result().numpy()+
            5*top3_metrics_obj.result().numpy()+
            3*top5_metrics_obj.result().numpy()+
            1*top10_metrics_obj.result().numpy())/19, 4),
          end='')
    endtime = time.time()

    test_total_loss /= tf.cast(batch_num, tf.float32)
    top1_metrics = top1_metrics_obj.result()
    top3_metrics = top3_metrics_obj.result()
    top5_metrics = top5_metrics_obj.result()
    top10_metrics = top10_metrics_obj.result()

    print('\r', 
        round(test_total_loss.numpy(), 4),
        round(top1_metrics.numpy(), 4),
        round(top3_metrics.numpy(), 4),
        round(top5_metrics.numpy(), 4),
        round(top10_metrics.numpy(), 4),
        round((10*top1_metrics.numpy()+
          5*top3_metrics.numpy()+
          3*top5_metrics.numpy()+
          1*top10_metrics.numpy())/19, 4),
        round(endtime-starttime, 4),
        len(test_image_path_list),
        round(len(test_image_path_list)/(endtime-starttime), 4),
        )

INFO: class_dict found and reused
model: se_resnext_101
loss: top1_metrics: top3_metrics: top5_metrics: top10_metrics: rwma_metrics: time_taken(s): sample_num speed(iter/s)
 1.9173 0.5203 0.7159 0.792 0.8742 0.6333 201.9888 7891 39.0665
